# 🏍️ MotoGP SQL Practice — Answers Version

In [ ]:
# Setup SQLite database for MotoGP Practice
!pip install ipython-sql sqlalchemy prettytable -q
%load_ext sql
%sql sqlite://
%config SqlMagic.style='PLAIN_COLUMNS'
%config SqlMagic.autopandas=True

%%sql
DROP TABLE IF EXISTS Team;
DROP TABLE IF EXISTS Rider;
DROP TABLE IF EXISTS Race;
DROP TABLE IF EXISTS Result;

CREATE TABLE Team (
  TeamID INTEGER PRIMARY KEY,
  Name TEXT,
  Country TEXT
);

CREATE TABLE Rider (
  RiderID INTEGER PRIMARY KEY,
  Name TEXT,
  Age INTEGER,
  Nationality TEXT,
  TeamID INTEGER,
  FOREIGN KEY(TeamID) REFERENCES Team(TeamID)
);

CREATE TABLE Race (
  RaceID INTEGER PRIMARY KEY,
  Name TEXT,
  Location TEXT,
  Date TEXT
);

CREATE TABLE Result (
  RiderID INTEGER,
  RaceID INTEGER,
  Position INTEGER,
  Points INTEGER,
  PRIMARY KEY (RiderID, RaceID),
  FOREIGN KEY(RiderID) REFERENCES Rider(RiderID),
  FOREIGN KEY(RaceID) REFERENCES Race(RaceID)
);

INSERT INTO Team VALUES
(1,'Ducati Lenovo Team','Italy'),
(2,'Monster Energy Yamaha','Japan'),
(3,'Repsol Honda','Japan'),
(4,'Red Bull KTM','Austria'),
(5,'Aprilia Racing','Italy');

INSERT INTO Rider VALUES
(1,'Francesco Bagnaia',27,'Italy',1),
(2,'Enea Bastianini',26,'Italy',1),
(3,'Fabio Quartararo',25,'France',2),
(4,'Marc Marquez',31,'Spain',3),
(5,'Joan Mir',28,'Spain',3),
(6,'Brad Binder',29,'South Africa',4),
(7,'Jack Miller',30,'Australia',4),
(8,'Maverick Viñales',29,'Spain',5),
(9,'Aleix Espargaro',34,'Spain',5),
(10,'Jorge Martin',26,'Spain',1);

INSERT INTO Race VALUES
(1,'Qatar Grand Prix','Lusail','2024-03-10'),
(2,'Spanish Grand Prix','Jerez','2024-04-28'),
(3,'Italian Grand Prix','Mugello','2024-06-02'),
(4,'Japanese Grand Prix','Motegi','2024-09-29'),
(5,'Malaysian Grand Prix','Sepang','2024-11-03'),
(6,'Valencia Grand Prix','Valencia','2024-11-17');

INSERT INTO Result VALUES
(1,1,1,25),(3,1,2,20),(4,1,3,16),(6,1,4,13),(10,1,5,11),
(1,2,2,20),(10,2,1,25),(4,2,3,16),(6,2,5,11),(8,2,4,13),
(3,3,1,25),(1,3,2,20),(9,3,3,16),(7,3,4,13),(4,3,5,11),
(10,4,1,25),(1,4,2,20),(3,4,3,16),(8,4,4,13),(6,4,5,11),
(1,5,1,25),(2,5,2,20),(3,5,3,16),(6,5,4,13),(4,5,5,11),
(3,6,1,25),(1,6,2,20),(10,6,3,16),(8,6,4,13),(9,6,5,11);


## Section A: Basics

In [ ]:
%%sql
SELECT Name, Nationality FROM Rider WHERE TeamID=1;

In [ ]:
%%sql
SELECT DISTINCT Country FROM Team;

In [ ]:
%%sql
SELECT * FROM Race WHERE Date BETWEEN '2024-04-01' AND '2024-08-01';

In [ ]:
%%sql
SELECT Name, Age FROM Rider WHERE Name LIKE 'F%';

## Section B: Joins & Aggregation

In [ ]:
%%sql
SELECT r.Name AS Rider, t.Name AS Team, SUM(res.Points) AS Total
FROM Result res JOIN Rider r ON res.RiderID=r.RiderID JOIN Team t ON r.TeamID=t.TeamID
GROUP BY r.RiderID ORDER BY Total DESC;

In [ ]:
%%sql
SELECT t.Name AS Team, AVG(res.Points) AS AvgPoints
FROM Result res JOIN Rider r ON res.RiderID=r.RiderID JOIN Team t ON r.TeamID=t.TeamID
GROUP BY t.TeamID HAVING AVG(res.Points)>15;

## Section C: Data Modification

In [ ]:
%%sql
INSERT INTO Rider VALUES(11,'Pedro Acosta',20,'Spain',4);
SELECT * FROM Rider WHERE Name='Pedro Acosta';

In [ ]:
%%sql
UPDATE Rider SET TeamID=2 WHERE Name='Marc Marquez';
SELECT Name, TeamID FROM Rider WHERE Name='Marc Marquez';

In [ ]:
%%sql
DELETE FROM Race WHERE Name='Malaysian Grand Prix';
SELECT * FROM Race;

## Section D: Advanced

In [ ]:
%%sql
SELECT r.Name, SUM(res.Points) AS TotalPoints,
CASE WHEN SUM(res.Points)>=100 THEN 'Elite'
     WHEN SUM(res.Points)>=70 THEN 'Competitive'
     ELSE 'Developing' END AS Category
FROM Result res JOIN Rider r ON res.RiderID=r.RiderID
GROUP BY r.Name ORDER BY TotalPoints DESC;

In [ ]:
%%sql
WITH team_total AS (SELECT t.TeamID, SUM(res.Points) AS TeamPoints FROM Result res JOIN Rider r ON res.RiderID=r.RiderID JOIN Team t ON r.TeamID=t.TeamID GROUP BY t.TeamID)
SELECT t.Name, TeamPoints FROM team_total t WHERE TeamPoints>(SELECT AVG(TeamPoints) FROM team_total);

In [ ]:
%%sql
BEGIN;
UPDATE Result SET Points=Points-5 WHERE RiderID=4;
SELECT RiderID, SUM(Points) FROM Result GROUP BY RiderID;
ROLLBACK;

## Section E: Challenges

In [ ]:
%%sql
-- 1) Leaderboard
SELECT r.Name,t.Name AS Team,SUM(res.Points) AS Points FROM Result res JOIN Rider r ON res.RiderID=r.RiderID JOIN Team t ON r.TeamID=t.TeamID GROUP BY r.RiderID ORDER BY Points DESC;

In [ ]:
%%sql
-- 2) Top Team
SELECT t.Name,SUM(res.Points) AS Total FROM Result res JOIN Rider r ON res.RiderID=r.RiderID JOIN Team t ON r.TeamID=t.TeamID GROUP BY t.TeamID ORDER BY Total DESC LIMIT 1;